In [1]:
import numpy as np
import argparse
import time
import cv2
import os
import matplotlib.pyplot as plt
import os
%matplotlib.pyplot as plt

UsageError: Line magic function `%matplotlib.pyplot` not found.


In [2]:
CONFIDENCE = 0.5
SCORE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.5

# the neural network configuration
config_path = "/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/object-detection-opencv-master/yolov3.cfg"
# the YOLO net weights file
weights_path = "/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/object-detection-opencv-master/yolov3.weights"
# weights_path = "weights/yolov3-tiny.weights"

# loading all the class labels (objects)
labels = open("/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/object-detection-opencv-master/darknet/data/coco.names").read().strip().split("\n")
# generating colors for each object for later plotting
colors = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")

In [3]:
# load the YOLO network
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)

In [4]:
count= 0
for i in os.listdir(os.path.join('/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/eccv14-data/data/images')):
    img = os.path.join('/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/eccv14-data/data/images',i)
    image = cv2.imread(img)
    file_name = os.path.basename(os.path.join('/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/eccv14-data/data/images',i))
    filename, ext = file_name.split(".")
    h, w = image.shape[:2]
    # create 4D blob
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    print("image.shape:", image.shape)
    print("blob.shape:", blob.shape)
    image.shape: (1200, 1800, 3)
    blob.shape: (1, 3, 416, 416)
    # sets the blob as the input of the network
    net.setInput(blob)
    # get all the layer names
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # feed forward (inference) and get the network output
    # measure how much it took in seconds
    start = time.perf_counter()
    layer_outputs = net.forward(ln)
    time_took = time.perf_counter() - start
    font_scale = 1
    thickness = 1
    boxes, confidences, class_ids = [], [], []
    # loop over each of the layer outputs
    for output in layer_outputs:
        # loop over each of the object detections
        for detection in output:
            # extract the class id (label) and confidence (as a probability) of
            # the current object detection
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            # discard out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > CONFIDENCE:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")
                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
        # perform the non maximum suppression given the scores defined before
        idxs = cv2.dnn.NMSBoxes(boxes, confidences, SCORE_THRESHOLD, IOU_THRESHOLD)
        # ensure at least one detection exists
        if len(idxs) > 0:
            # loop over the indexes we are keeping
            for i in idxs.flatten():
                # extract the bounding box coordinates
                x, y = boxes[i][0], boxes[i][1]
                w, h = boxes[i][2], boxes[i][3]
                # draw a bounding box rectangle and label on the image
                color = [int(c) for c in colors[class_ids[i]]]
                cv2.rectangle(image, (x, y), (x + w, y + h), color=color, thickness=thickness)
                text = f"{labels[class_ids[i]]}: {confidences[i]:.2f}"
                # calculate text width & height to draw the transparent boxes as background of the text
                (text_width, text_height) = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
                text_offset_x = x
                text_offset_y = y - 5
                box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))
                overlay = image.copy()
                cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
                # add opacity (transparency to the box)
                image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
                # now put the text (label: confidence %)
                cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=font_scale, color=(0, 0, 0), thickness=thickness)
        
#         cv2.imwrite(''+filename + "_yolo3." + ext, image)
        cv2.imwrite(os.path.join('/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/object-detection-opencv-master/object_detected/' , filename + "_yolo3." + ext), image)
        cv2.waitKey(0)
    count = count+1
    print(count)


image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
2
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
3
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
4
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
5
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
6
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
7
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
8
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
9
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
10
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
11
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
12
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
13
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
14
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
15
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
16
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
17
image.

139
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
140
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
141
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
142
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
143
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
144
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
145
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
146
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
147
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
148
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
149
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
150
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
151
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
152
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
153
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
154
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
155
image.shape: (425, 560, 3)
blob.shap

276
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
277
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
278
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
279
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
280
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
281
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
282
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
283
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
284
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
285
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
286
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
287
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
288
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
289
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
290
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
291
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
292
image.shape: (425, 560, 3)
blob.shap

413
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
414
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
415
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
416
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
417
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
418
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
419
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
420
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
421
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
422
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
423
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
424
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
425
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
426
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
427
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
428
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
429
image.shape: (425, 560, 3)
blob.shap

550
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
551
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
552
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
553
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
554
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
555
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
556
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
557
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
558
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
559
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
560
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
561
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
562
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
563
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
564
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
565
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
566
image.shape: (425, 560, 3)
blob.shap

...

824
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
825
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
826
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
827
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
828
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
829
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
830
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
831
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
832
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
833
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
834
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
835
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
836
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
837
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
838
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
839
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
840
image.shape: (425, 560, 3)
blob.shap

961
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
962
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
963
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
964
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
965
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
966
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
967
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
968
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
969
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
970
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
971
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
972
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
973
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
974
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
975
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
976
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
977
image.shape: (425, 560, 3)
blob.shap

1096
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1097
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1098
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1099
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1100
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1101
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1102
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1103
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1104
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1105
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1106
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1107
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1108
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1109
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1110
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1111
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1112
image.shape: (425, 

1231
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1232
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1233
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1234
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1235
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1236
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1237
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1238
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1239
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1240
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1241
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1242
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1243
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1244
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1245
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1246
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1247
image.shape: (425, 

1366
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1367
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1368
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1369
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1370
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1371
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1372
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1373
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1374
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1375
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1376
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1377
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1378
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1379
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1380
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1381
image.shape: (425, 560, 3)
blob.shape: (1, 3, 416, 416)
1382
image.shape: (425, 

In [14]:
# path_name = "/media/sagar/HDD1/IIIT_PGSSP/CV/Project/MY_OBJ_DETECTION_Y/object-detection-opencv-master/img_5001.png"
# image = cv2.imread(path_name)
# file_name = os.path.basename(path_name)
# filename, ext = file_name.split(".")